In [ ]:
# default_exp calibrate

# Calibration Procedure

> summary

Every OpenHSI camera is unique and requires calibration before use. This module provides the abstractions to create the calibration data which are then used in operation. 


In [ ]:
#hide

# documentation extraction for class methods
from nbdev.showdoc import *

# unit tests using test_eq(...)
from fastcore.test import *

# monkey patching class methods using @patch
from fastcore.foundation import *
from fastcore.foundation import patch

# imitation of Julia's multiple dispatch using @typedispatch
from fastcore.dispatch import typedispatch

# bring forth **kwargs from an inherited class for documentation
from fastcore.meta import delegates

In [ ]:
#export

from fastcore.foundation import patch
from fastcore.meta import delegates
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d
from PIL import Image
from scipy.signal import decimate

from typing import Iterable, Union, Callable, List, TypeVar, Generic, Tuple, Optional
import datetime
import json
import pickle

In [ ]:
#export

from openhsi.data import *
from openhsi.capture import *
from openhsi.cameras import *

In [ ]:

"""
cam_prop.settings["longitude"] = -17.7
cam_prop.settings["latitude"] = 146.1
cam_prop.settings["datetime_str"] = "2021-05-26 03:26"
cam_prop.settings["altitude"] = 0.12
cam_prop.settings["radiosonde_station_num"] = 94299
cam_prop.settings["radiosonde_region"] = "pac"
cam_prop.settings["sixs_path"] = "assets/sixsV1.1"
"""

## Find illuminated sensor area

Since the longer dimension is used for the spectral channels, the rows correspond to the cross-track dimension and are limited by the optics (slit). The useable area is cropped out.

In [ ]:
#export

class SettingsBuilderMixin():
    
    def say_hello(self,):
        print("Hello!")

    def row_minmax(self, img:np.ndarray, show:bool=False) -> tuple:
        col_summed = np.sum(img,axis=1)
        edges = np.abs(np.gradient(col_summed))
        locs = find_peaks(edges, height=5000, width=1.5, prominence=0.01)[0]
        if show:
            plt.plot(col_summed)
            plt.plot([locs[0],locs[0]],[0,np.max(col_summed)],'r',alpha=0.5,label=f"{locs[0]}")
            plt.plot([locs[-1],locs[-1]],[0,np.max(col_summed)],'r',alpha=0.5,label=f"{locs[-1]}")
            plt.xlabel("row index")
            plt.legend()
        return (int(locs[0]),int(locs[-1]))
    

## Smile Correction

The emissions lines, which should be straight vertical, appear slightly curved. This is smile error (error in the spectral dimension). 

In [ ]:
#export

@patch
def find_smile_shifts(self:SettingsBuilderMixin, img:np.ndarray) -> np.array:
    sz = np.shape(img)
    window = np.int32(np.flip(img[202,:].copy()))
    
    shifts = np.zeros((sz[0],),dtype=np.int16)
    
    for i in range(sz[0]):
        pattern_match = np.convolve(img[i,:],window,"same")
        shifts[i] = np.argmax(pattern_match)
    
    shifts -= sz[1]//2
    shifts -= np.min(shifts) # make all entries positive
    return shifts

## Map the spectral axis to wavelengths

To do this, peaks in the HgAr spectrum are found, refined by curve-fitting with Gaussians. The location of the peaks then allow for interpolation to get the map from array (column) index to wavelength (nm).

In [ ]:
#export

# top amplitude emission lines sorted in ascending order. You can use fewer entries if you'd like.
HgAr_lines = np.array([404.656,435.833,546.074,576.960,579.066,696.543,738.393,
                           750.387,763.511,772.376,794.818,800.616,811.531])

def sum_gaussians(x:"indices np.array", 
                    *args:"amplitude, peak position, peak width, constant") -> np.array:
    split = len(args)//3
    A   = args[0:split]         # amplitude
    mu  = args[split:2*split]   # peak position
    sigma = args[split*2:-1]    # peak stdev
    c   = args[-1]              # offset
    return np.array( [A[i] * np.exp( - np.square( (x - mu[i])/sigma[i] ) ) 
                        for i in range(len(A))] ).sum(axis=0) + c

from scipy.signal import savgol_filter

@patch
def fit_spectral_lines(self:SettingsBuilderMixin, spectra:np.array,spectral_lines:list,show=True):
    """finds the index to wavelength map given a spectra and a list of emission lines."""
    
    filtered_spec = savgol_filter(spectra, 5, 3)
    
    mu, props = find_peaks(filtered_spec, height = 40, width = 1.5, prominence = 0.1)
    A = props["peak_heights"] # amplitude
    σ = 0.5 * props["widths"] # standard deviation
    c = 0.02                  # constant
    params0 = [*A,*mu,*σ,c]   # flatten to 1D array
    
    # refine the estimates from find_peaks by curve fitting Gaussians
    coeffs, _ = curve_fit(sum_gaussians, np.arange(len(spectra)), spectra, p0=params0)
    
    split = len(params0)//3
    A = coeffs[:split]
    μ = coeffs[split:2*split]
    σ = coeffs[2*split:-1]
    
    # find the array index for the top amplitude emissions lines
    top_A_idx = np.flip(np.argsort(A))[:len(spectral_lines)]
    sorted_idx = np.sort(μ[top_A_idx])
    
    # calculate the wavelength corresponding to each array index (should be straight)
    poly_func = np.poly1d( np.polyfit(sorted_idx, spectral_lines, 2) )
    wavelengths = poly_func(np.arange(len(spectra)))
    
    if show:
        plt.subplots(figsize=(12,3))
        plt.plot(wavelengths,spectra)
        plt.xlabel("wavelength (nm)")
        plt.ylabel("digital number")
        for i in np.uint16(np.round(sorted_idx)):
            plt.plot([wavelengths[i],wavelengths[i]],[0,np.max(spectra)],'r',alpha=0.5)
            print(wavelengths[i])
        plt.axis([min(wavelengths),max(wavelengths),0,max(spectra)])
        
        #plt.show()
    
    return wavelengths

In [ ]:

#export

class SettingsBuilderMetaclass(type):
    def __new__(cls, clsname:str, cam_class, attrs) -> "Class":
        """Create a SettingsBuilder class based on your chosen `CameraClass`."""
        return super(SettingsBuilderMetaclass, cls).__new__(cls, clsname, (cam_class,SettingsBuilderMixin), attrs)
    

def create_settings_builder(clsname:str, cam_class:"Camera Class") -> "SettingsBuilder Class":
    """Create a `SettingsBuilder` class called `clsname` based on your chosen `cam_class`."""
    return type(clsname, (cam_class,SettingsBuilderMixin), {})
    
    

In [ ]:


SettingsBuilder = create_settings_builder("SettingsBuilder",SimulatedCamera)
sb = SettingsBuilder(txt_path="assets/cam_settings.txt",pkl_path="assets/cam_calibration.pkl")
sb.say_hello()

Hello!



@delegates()
class SettingsBuilder(CameraProperties):
    
    def __init__(self,cam_class:"custom camera class" = SimulatedCamera, **kwargs):
        """"""
        super().__init__(**kwargs)
        
        self.cam_class = cam_class
        with cam_class(**kwargs) as cam:
            cam.start_cam()
            img = cam.get_img()
            cam.stop_cam()
            
        
        

sb = SettingsBuilder(img_path="assets/rocky_beach.png", n_lines=1024, processing_lvl = 3)

SB2 = type("SB2",(SimulatedCamera,),{"hello":2})
sb2 = SB2()
sb2.hello